# Fine tune large language model

Within this challenge, we are tasked with the development of AI systems capable of receiving textual descriptions as input and generating high-quality audio wave files as output. These AI systems will craft customized background music, considering various elements such as melody, hits, styles, and more to evoke the intended emotional and contextual resonance.

Using example from this website:
https://docs.ray.io/en/latest/train/examples/lightning/vicuna_13b_lightning_deepspeed_finetune.html

# Installation and Setup Environment

In [ ]:
%%capture
!pip3 install ray
!pip3 install datasets
!pip3 install transformer
!pip3 install numpy datasets "transformers>=4.19.1" "pytorch_lightning>=1.6.5"
!pip3 install accelerate
!pip3 install lightning
!pip3 install deepspeed
!pip3 install fad_pytorch

In [2]:
import ray
import re
import ray
import json
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForQuestionAnswering
from datasets import concatenate_datasets, load_dataset

NUM_WORKERS = 2
BATCH_SIZE_PER_WORKER = 8
MODEL_NAME = "gpt2"

In [ ]:
ray.shutdown()
ray.init(
    runtime_env={
        "pip": [
            "datasets==2.13.1",
            "torch>=1.13.0",
            "deepspeed==0.9.4",
            "accelerate>=0.20.3",
            "transformers==4.30.2",
            "lightning==2.0.3",
        ],
    },
    ignore_reinit_error=True,
)

# Get Dataset and preprocessing

get dataset from json file

In [ ]:
from datasets import Dataset, DatasetDict

# Create a list of dictionaries where each dictionary represents a sample in the dataset
with open("/Users/khoavo2003/Documents/GitHub/muze/new_train.json", 'r') as json_file:
    data = json.load(json_file)

# reformat data
dict_dataset = {"description":[], "url": []}
for info in data.values():
  dict_dataset['description'].append(info['description'])
  dict_dataset['url'].append(info['url'])

# Create a datasets.Dataset instance
# You can also specify additional metadata such as features and split
my_dataset = Dataset.from_dict(dict_dataset, split='train')

# Save the dataset to a file (optional)
# dataset_dict.save_to_disk("my_dataset")

# Load the dataset from the saved file
# loaded_dataset = DatasetDict.load_from_disk("my_dataset")


In [27]:
my_dataset

Dataset({
    features: ['description', 'url'],
    num_rows: 10000
})

preprocess input and label to appropriate format

In [ ]:
#TODO: include the tokenizer for input and output, and code to get url
tokenizer_input = AutoTokenizer.from_pretrained('gpt2')
tokenizer_output = 
def preprocess_input_output(input: str, output: str):
  '''
  function to preprocess input based on input and output tokenizer
  args:
    - input(str): string need to encode (description in this case)
    - output(str): url to the music

  returns:
    - input_preprocess: list of integer
    - output_preprocess: list of integer
  '''
  input_token = tokenizer_input.tokenize(input)
  input_preprocess = tokenizer_input.convert_tokens_to_ids(input_token)
  #TODO: include code to get data from url, change it to appropriate format, and get preprocess output
  output_token = tokenizer_output.tokenize(output)
  output_preprocess = tokenizer_output.convert_tokens_to_ids(output_token)
  return input_preprocess, output_preprocess

In [ ]:
processed_dataset = my_dataset.map(
    lambda example: preprocess_input_output(example['description'], example['url']),
    batched=True,
    remove_columns=['description', 'url'],  # Remove the original columns
)
# TODO: Rename columns to 'input_ids' and 'labels'
processed_dataset = processed_dataset.rename_column("0", "input_ids")
processed_dataset = processed_dataset.rename_column("1", "labels")
processed_dataset = processed_dataset.train_test_split(train_size=0.8, seed=20)
processed_dataset["validation"] = processed_dataset.pop("test")

split into train and evaluation set

# Preprocessing

change audio data into format that can be used to train and evaluate the model 

tokenize music 


# Model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# get pretrain model 
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

 define a collate function that will apply the correct amount of padding to the items of the dataset we want to batch together.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer_input, model=model)

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"result",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
)

create dataloader

In [ ]:
from torch.utils.data import DataLoader

# Training DataLoader
train_dataloader = DataLoader(
    processed_dataset["train"],            # Training dataset
    shuffle=True,                          # Shuffle the data at each epoch
    batch_size=8,                          # Number of samples in each batch
    collate_fn=data_collator,              # Collate function to process batches
    num_workers=4,                         # Number of subprocesses to use for data loading
    pin_memory=True                        # Pin memory for faster data transfer to GPU
)

# Evaluation DataLoader
eval_dataloader = DataLoader(
    processed_dataset["validation"],       # Validation dataset
    batch_size=8,                          # Number of samples in each batch
    collate_fn=data_collator,              # Collate function to process batches
    num_workers=4,                         # Number of subprocesses to use for data loading
    pin_memory=True                        # Pin memory for faster data transfer to GPU
)


# Evaluation

https://github.com/msight-tech/research-fad
https://github.com/LAION-AI/CLAP

# Training

In [ ]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="result",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=args,
    data_collator=data_collator,
    train_dataset=processed_dataset["train"],
    eval_dataset=processed_dataset["valid"],
)

In [ ]:
trainer.train()

Another code for training part

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)

# Demonstration

In [ ]:
from transformers import AutoModelForSeq2SeqLM

# get pretrain model 
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)

# Strach (for testing code)

In [2]:
import librosa
from IPython.display import Audio, display, clear_output
import ipywidgets as widgets

# Get the file path to an included audio example
filename = "we_wish_you.mp3"

# Load the audio as a waveform `y`
# Store the sampling rate as `sr`
y, sr = librosa.load(filename)

# Run the default beat tracker
tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

# Convert the frame indices of beat events into timestamps
beat_times = librosa.frames_to_time(beat_frames, sr=sr)

# Function to play audio when the button is clicked
clear_output(wait=True)  # Clear the previous output, if any
display(Audio(y, rate=sr))  # Play the audio

Estimated tempo: 184.57 beats per minute


https://www.analyticsvidhya.com/blog/2023/09/text-to-sound-train-your-large-language-models/

In [7]:
from miditok import REMI, TokenizerConfig  # here we choose to use REMI

# Our parameters
TOKENIZER_PARAMS = {
    "pitch_range": (21, 109),
    "beat_res": {(0, 4): 8, (4, 12): 4},
    "nb_velocities": 32,
    "special_tokens": ["PAD", "BOS", "EOS", "MASK"],
    "use_chords": True,
    "use_rests": False,
    "use_tempos": True,
    "use_time_signatures": False,
    "use_programs": False,
    "nb_tempos": 32,  # nb of tempo bins
    "tempo_range": (40, 250),  # (min, max)
}
config = TokenizerConfig(**TOKENIZER_PARAMS)

# Creates the tokenizer
tokenizer = REMI(config)

In [28]:
from miditoolkit import MidiFile

# Tokenize a MIDI file
midi = MidiFile("/Users/khoavo2003/Documents/GitHub/muze/midi_dataset/we_wish_you.mid")
tokens = tokenizer(midi)  # automatically detects MidiFile, paths

In [32]:
tokens[0].ids

[4,
 200,
 58,
 105,
 127,
 206,
 15,
 111,
 143,
 27,
 109,
 133,
 63,
 105,
 129,
 211,
 63,
 108,
 127,
 216,
 63,
 106,
 127,
 219,
 62,
 107,
 127,
 4,
 189,
 60,
 105,
 130,
 191,
 20,
 112,
 138,
 193,
 8,
 109,
 134,
 195,
 60,
 108,
 129,
 200,
 60,
 106,
 134,
 205,
 65,
 106,
 130,
 206,
 17,
 114,
 139,
 211,
 65,
 108,
 126,
 214,
 60,
 105,
 128,
 67,
 111,
 126,
 219,
 63,
 107,
 126,
 4,
 189,
 22,
 112,
 140,
 62,
 109,
 128,
 192,
 10,
 109,
 136,
 195,
 58,
 113,
 129,
 200,
 58,
 109,
 137,
 205,
 19,
 111,
 140,
 206,
 67,
 110,
 129,
 210,
 7,
 107,
 133,
 211,
 67,
 110,
 126,
 214,
 68,
 106,
 128,
 218,
 12,
 102,
 127,
 219,
 65,
 107,
 126,
 4,
 189,
 12,
 112,
 144,
 63,
 105,
 129,
 190,
 24,
 113,
 143,
 194,
 60,
 110,
 130,
 200,
 58,
 112,
 127,
 203,
 58,
 109,
 130,
 205,
 17,
 115,
 135,
 60,
 105,
 134,
 211,
 65,
 108,
 129,
 212,
 58,
 103,
 131,
 216,
 22,
 111,
 129,
 62,
 106,
 127,
 4,
 189,
 15,
 112,
 139,
 190,
 27,
 108,
 138,
 201,
 39,
 

In [33]:
real_tokens = tokenizer._ids_to_tokens(tokens[0].ids)

In [35]:
out_midi = tokenizer([real_tokens])

In [37]:
out_midi.dump('we_wish_you123.mid')

In [ ]:
from miditok import REMI, TokSequence
from copy import deepcopy

tokenizer = REMI()  # using defaults parameters (constants.py)
tokens_no_bpe_paths = list(Path('path', 'to', 'dataset').glob('**/*.json'))


In [26]:
tokens

[TokSequence(tokens=['Bar_None', 'Position_0', 'Tempo_121.29', 'Position_11', 'Pitch_74', 'Velocity_51', 'Duration_0.3.8', 'Position_17', 'Pitch_31', 'Velocity_75', 'Duration_2.3.8', 'Pitch_43', 'Velocity_67', 'Duration_1.1.8', 'Pitch_79', 'Velocity_51', 'Duration_0.5.8', 'Position_22', 'Pitch_79', 'Velocity_63', 'Duration_0.3.8', 'Position_27', 'Pitch_79', 'Velocity_55', 'Duration_0.3.8', 'Position_30', 'Pitch_78', 'Velocity_59', 'Duration_0.3.8', 'Bar_None', 'Position_0', 'Pitch_76', 'Velocity_51', 'Duration_0.6.8', 'Position_2', 'Pitch_36', 'Velocity_79', 'Duration_1.6.8', 'Position_4', 'Pitch_24', 'Velocity_67', 'Duration_1.2.8', 'Position_6', 'Pitch_76', 'Velocity_63', 'Duration_0.5.8', 'Position_11', 'Pitch_76', 'Velocity_55', 'Duration_1.2.8', 'Position_16', 'Pitch_81', 'Velocity_55', 'Duration_0.6.8', 'Position_17', 'Pitch_33', 'Velocity_87', 'Duration_1.7.8', 'Position_22', 'Pitch_81', 'Velocity_63', 'Duration_0.2.8', 'Position_25', 'Pitch_76', 'Velocity_51', 'Duration_0.4.8',